In [15]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine

In [3]:
df = pd.read_csv('D://code//data//data_SA_north//all.csv', low_memory=False)

In [13]:
df_taian = pd.read_csv('D://code//data//data_SA_north//Taian.csv', low_memory=False)

In [4]:
df['ENTRY_TIME'] = pd.to_datetime(df['ENTRY_TIME'],infer_datetime_format=True)
df['EXIT_TIME'] = pd.to_datetime(df['EXIT_TIME'],infer_datetime_format=True)
df['O_TIME'] = pd.to_datetime(df['O_TIME'],infer_datetime_format=True)
df['D_TIME'] = pd.to_datetime(df['D_TIME'],infer_datetime_format=True)

In [2]:
start_min = [00, 11, 21, 31, 41, 51]
end_min = [10, 20, 30, 40, 50, 59]
directions = ['N', 'S']
days = [21, 22, 23, 24, 25, 26, 27]
categories = [31, 32, 41, 42, 5]
hours = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
small_car = [31, 32]
big_car = [42, 5]
big_41 = [41]
n_cats = [small_car, big_41, big_car]


In [5]:
gnty_cnty_df = pd.read_excel('C://Users//AthenaLee//Documents//code//data//data_SA//GATRY_CITY.xlsx')

In [4]:
gnty_df = pd.read_csv('C://Users//AthenaLee//Documents//code//data//data_SA//GANTRY_NAME.csv', low_memory=False)

In [3]:
def make_clean_gantry(x):
    x = x.strip()
    return x

In [6]:
gnty_cnty_df ['GANTRY_ID'] = gnty_cnty_df ['GANTRY_ID'].apply(make_clean_gantry)
gnty_cnty_df ['CODE_NAME'] = gnty_cnty_df ['CODE_NAME'].apply(make_clean_gantry)
gnty_df['GANTRY_ID'] = gnty_df['GANTRY_ID'].apply(make_clean_gantry)

In [ ]:
df['in_location'] = df['O_CITY'].apply(define_gantry_location_in)
df['out_location'] = df['D_CITY'].apply(define_gantry_location_out)

In [7]:
def split_gantry_in(x) :
    var = x.split('-')[0]
    return var

In [8]:
def split_gantry_out(x) :
    var = x.split('-')[1]
    return var

In [23]:
gnty_df['GANTRY_ID'] = gnty_df['GANTRY_ID'].apply(make_clean_gantry)

In [9]:
gnty_df["in"] = gnty_df['GANTRY_NAME'].apply(split_gantry_in)
gnty_df["out"] = gnty_df['GANTRY_NAME'].apply(split_gantry_out)

In [11]:
# for each SA df, apply to ["O_CITY"]
def define_gantry_location_in(x):
    #location = ''
    for index, gantry in enumerate(gnty_df['GANTRY_ID']):
        if x == gantry :
            location = gnty_df.iloc[index, :]['in']
    return location        

In [13]:
# for each SA df, apply to ["D_CITY"]
def define_gantry_location_out(x):
    #location = ''
    for index, gantry in enumerate(gnty_df['GANTRY_ID']):
        if x == gantry :
            location = gnty_df.iloc[index, :]['out']
    return location     

In [14]:
# for each SA df, apply to ["O_CITY"]
def define_gantry_location_country(x):
    #location = ''
    for index, gantry in enumerate(gnty_cnty_df ['GANTRY_ID']):
        if x == gantry :
            location = gnty_cnty_df.iloc[index, :]['CODE_NAME']
    return location        

In [15]:
df_taian['O_COUNTRY'] = df_taian['O_CITY'].apply(define_gantry_location_country)

In [26]:
df['in_location'] = df['O_CITY'].apply(define_gantry_location_in)
df['out_location'] = df['D_CITY'].apply(define_gantry_location_out)
df['O_COUNTRY'] = df['O_CITY'].apply(define_gantry_location_country)
df['D_COUNTRY'] = df['D_CITY'].apply(define_gantry_location_country)

In [29]:
df.to_csv('D://code//data//data_SA_north//all_wOD.csv')

#### combine all data with OD country and OD gateway, then insert into SQL Server

In [16]:
# latest
engine = create_engine('mssql://ATHENALEE-PC\\SQLEXPRESS/athena_dev?driver=SQL+Server?trusted_connection=yes') 

In [17]:
SA_name = ["Taian", "Zhongli", "Xihu", "Xilou", "Nantou", "QingShui", "Huko", "Guanxi"]
df_dicts = {}
df_lists = []
for sa in SA_name :
    if sa == "Zhongli" :
        df_N = pd.read_csv('C://Users//AthenaLee//Documents//code//data//data_SA//Zhongli_N.csv', low_memory=False)
        df_S = pd.read_csv('C://Users//AthenaLee//Documents//code//data//data_SA//Zhongli_S.csv', low_memory=False)
        df = df = pd.concat([df_N, df_S])       
    else :
        df = pd.read_csv('C://Users//AthenaLee//Documents//code//data//data_SA//{0}.csv'.format(sa), low_memory=False)
    print("{0} : done loading data ...".format(sa))
    
    df['ENTRY_TIME'] = pd.to_datetime(df['ENTRY_TIME'],infer_datetime_format=True)
    df['EXIT_TIME'] = pd.to_datetime(df['EXIT_TIME'],infer_datetime_format=True)
    df['O_TIME'] = pd.to_datetime(df['O_TIME'],infer_datetime_format=True)
    df['D_TIME'] = pd.to_datetime(df['D_TIME'],infer_datetime_format=True)
    print("{0} : done formating time columns ...".format(sa))
    
    df['in_location'] = df['O_CITY'].apply(define_gantry_location_in)
    print("{0} : done OD columns <in_location> ...".format(sa))
    df['out_location'] = df['D_CITY'].apply(define_gantry_location_out)
    print("{0} : done OD columns <out_location> ...".format(sa))
    df['O_COUNTRY'] = df['O_CITY'].apply(define_gantry_location_country)
    print("{0} : done OD columns <O_COUNTRY> ...".format(sa))
    df['D_COUNTRY'] = df['D_CITY'].apply(define_gantry_location_country)
    print("{0} : done OD columns <D_COUNTRY> ...".format(sa))
    
    df_dicts['{0}_df'.format(sa)] = df
    df_lists.append(df_dicts['{0}_df'.format(sa)])
    print("{0} : done ! ...".format(sa))
final_df = pd.concat(df_lists)
print('=========== done concat ! ===========')
final_df.to_sql("all_wOD", engine, if_exists='replace', chunksize=50)
print('inserted successfully !')

Taian : done loading data ...
Taian : done formating time columns ...
Taian : done OD columns <in_location> ...
Taian : done OD columns <out_location> ...
Taian : done OD columns <O_COUNTRY> ...
Taian : done OD columns <D_COUNTRY> ...
Taian : done ! ...
Zhongli : done loading data ...
Zhongli : done formating time columns ...
Zhongli : done OD columns <in_location> ...
Zhongli : done OD columns <out_location> ...
Zhongli : done OD columns <O_COUNTRY> ...
Zhongli : done OD columns <D_COUNTRY> ...
Zhongli : done ! ...
Xihu : done loading data ...
Xihu : done formating time columns ...
Xihu : done OD columns <in_location> ...
Xihu : done OD columns <out_location> ...
Xihu : done OD columns <O_COUNTRY> ...
Xihu : done OD columns <D_COUNTRY> ...
Xihu : done ! ...
Xilou : done loading data ...
Xilou : done formating time columns ...
Xilou : done OD columns <in_location> ...
Xilou : done OD columns <out_location> ...
Xilou : done OD columns <O_COUNTRY> ...
Xilou : done OD columns <D_COUNTRY> 

In [18]:
final_df.to_csv('C://Users//AthenaLee//Documents//code//data//data_SA//all_wOD.csv')

In [32]:
16726.674238204956/60/60
168055/60/60

46.68194444444444

In [29]:
import time
start_time_out = time.time()
sorting_trip(final_df)
print("---query time : %s seconds ---" % (time.time() - start_time_out))

---bms calculation : 16727.963806152344 seconds ---
done calculation !
---泰安 category 31 : 16747.52889752388 seconds ---
done 泰安 31 file
---bms calculation : 4673.592910528183 seconds ---
done calculation !
---泰安 category 32 : 4692.383670806885 seconds ---
done 泰安 32 file
---bms calculation : 1149.0032329559326 seconds ---
done calculation !
---泰安 category 41 : 1165.1064653396606 seconds ---
done 泰安 41 file
---bms calculation : 1204.7235004901886 seconds ---
done calculation !
---泰安 category 42 : 1221.292324066162 seconds ---
done 泰安 42 file
---bms calculation : 801.8024010658264 seconds ---
done calculation !
---泰安 category 5 : 817.7492694854736 seconds ---
done 泰安 5 file
---bms calculation : 5580.434569597244 seconds ---
done calculation !
---中壢 category 31 : 5598.810527801514 seconds ---
done 中壢 31 file
---bms calculation : 2006.5927443504333 seconds ---
done calculation !
---中壢 category 32 : 2021.7653274536133 seconds ---
done 中壢 32 file
---bms calculation : 422.2356231212616 secon

In [26]:
def sorting_trip(df) :
    SA_name = ["Taian", "Zhongli", "Xihu", "Xilou", "Nantou", "QingShui", "Huko", "Guanxi"]
    SA_Cname = ['泰安', '中壢', '西湖', '西螺', '南投', '清水', '湖口', '關西']
    categories = [31, 32, 41, 42, 5]
    for idx, sa in enumerate(SA_Cname) :  # the first SA
        #writer = pd.ExcelWriter('C://Users//Administrator//Documents//code//data//SA_advanced//analysis_result//{0}_{1}_all_trip.xlsx'.format(sa, category))
        categories_df = {}
        
        for category in categories :
            start_time = time.time()
            #writer = pd.ExcelWriter('C://Users//Administrator//Documents//code//data//SA_advanced//analysis_result//{0}_{1}_all_trip.xlsx'.format(sa, category))
            var_dict = {}
            cnt_row = []
            for var in SA_Cname :
                var_dict['v_{0}'.format(var)] = 0
            
            unique_bms = df[(df.SERVICE_AREA == sa)&(df.STOP == 'P')&(df.MVDIS_CATEGORY == category)]['BMS_TX_BATCH'].unique()
            
            first_row = []
            second_row = []
            third_row = []
            col_name = []
            
            for bms in unique_bms :
                series = df[(df.BMS_TX_BATCH == bms)&(df.STOP == 'P')].groupby("SERVICE_AREA").size()
                cnt_df = pd.DataFrame(series).reset_index()
                length = cnt_df.shape[0]
                for p_name in [x for index, x in enumerate(SA_Cname) if index != idx] : # this list won't have the first SA itself 
                    for idx_cnt, i in cnt_df.iterrows():
                        if i.SERVICE_AREA == p_name :
                            var_dict["v_{0}".format(p_name)] = var_dict["v_{0}".format(p_name)] + i[0]
            print("---bms calculation : %s seconds ---" % (time.time() - start_time))
            print('done calculation !')
            # trip grouping start here
            #bms_df = df[(df.BMS_TX_BATCH.isin(unique_bms))]
            for sa2 in [x for index, x in enumerate(SA_Cname) if index != idx] : # sa2 should be the same with p_name
                col_name.append(sa2)
                select_vID = df[(df.BMS_TX_BATCH.isin(unique_bms))&(df.SERVICE_AREA == sa2)&(df.STOP == 'P')]['VEHICLE_ID'].unique()
                trip_series = df[df.VEHICLE_ID.isin(select_vID)].groupby(['in_location', 'out_location']).size().sort_values(0, ascending=False)[:3]
                trip_df = pd.DataFrame(trip_series).reset_index()
                trip_dict = {}
                for idx_trip, place in trip_df.iterrows():
                    place_string = "{0}-{1}".format(place['in_location'], place['out_location'])
                    trip_dict['{0}_place'.format(idx_trip)] = place_string
                    if idx_trip == 0 :
                        first_row.append(trip_dict['{0}_place'.format(idx_trip)])
                    elif idx_trip == 1 :
                        second_row.append(trip_dict['{0}_place'.format(idx_trip)])
                    elif idx_trip == 2 :
                        third_row.append(trip_dict['{0}_place'.format(idx_trip)])
                # collect the variables of car counting
                cnt_row.append(var_dict['v_{0}'.format(sa2)])
            result_df =  pd.DataFrame([cnt_row, first_row, second_row, third_row], columns = col_name)    
            categories_df['{0}_{1}'.format(sa, category)] = result_df
            
            writer = pd.ExcelWriter('C://Users//AthenaLee//Documents//code//data//data_SA//analysis_result\ALL_TRIP_SORTING//{0}_{1}_all_trip.xlsx'.format(sa, category))
            categories_df['{0}_{1}'.format(sa, category)].to_excel(writer,'{0}_{1}_trip'.format(sa, category)) 
            writer.save()
            print("---{0} category {1} : {2} seconds ---" .format(sa, category,(time.time() - start_time)))
            print('done {0} {1} file'.format(sa, category))
        # saving excel
        #writer = pd.ExcelWriter('C://Users//Administrator//Documents//code//data//SA_advanced//analysis_result//{0}_{1}_all_trip.xlsx'.format(sa, category))
        #writer.save()

In [ ]:
SA_Cname = ['泰安']
categories = [31, 32, 41, 42, 5]
    for idx, sa in enumerate(SA_Cname) :  # the first SA
        #writer = pd.ExcelWriter('C://Users//Administrator//Documents//code//data//SA_advanced//analysis_result//{0}_{1}_all_trip.xlsx'.format(sa, category))
        categories_df = {}
        
        for category in categories :
            #writer = pd.ExcelWriter('C://Users//Administrator//Documents//code//data//SA_advanced//analysis_result//{0}_{1}_all_trip.xlsx'.format(sa, category))
            var_dict = {}
            cnt_row = []
            for var in SA_Cname :
                var_dict['v_{0}'.format(var)] = 0
            
            unique_bms = df[(df.SERVICE_AREA == sa)&(df.STOP == 'P')&(df.MVDIS_CATEGORY == category)]['BMS_TX_BATCH'].unique()
            
            first_row = []
            second_row = []
            third_row = []
            col_name = []
            
            for bms in unique_bms :
                series = df[(df.BMS_TX_BATCH == bms)&(df.STOP == 'P')].groupby("SERVICE_AREA").size()
                cnt_df = pd.DataFrame(series).reset_index()
                length = cnt_df.shape[0]
                for p_name in [x for index, x in enumerate(SA_Cname) if index != idx] : # this list won't have the first SA itself 
                    for idx_cnt, i in cnt_df.iterrows():
                        if i.SERVICE_AREA == p_name :
                            var_dict["v_{0}".format(p_name)] = var_dict["v_{0}".format(p_name)] + i[0]
            # trip grouping start here
            #bms_df = df[(df.BMS_TX_BATCH.isin(unique_bms))]
            for sa2 in [x for index, x in enumerate(SA_Cname) if index != idx] : # sa2 should be the same with p_name
                col_name.append(sa2)
                select_vID = df[(df.BMS_TX_BATCH.isin(unique_bms))&(df.SERVICE_AREA == sa2)&(df.STOP == 'P')]['VEHICLE_ID'].unique()
                trip_series = df[df.VEHICLE_ID.isin(select_vID)].groupby(['in_location', 'out_location']).size().sort_values(0, ascending=False)[:3]
                trip_df = pd.DataFrame(trip_series).reset_index()
                trip_dict = {}
                for idx_trip, place in trip_df.iterrows():
                    place_string = "{0}-{1}".format(place['in_location'], place['out_location'])
                    trip_dict['{0}_place'.format(idx_trip)] = place_string
                    if idx_trip == 0 :
                        first_row.append(trip_dict['{0}_place'.format(idx_trip)])
                    elif idx_trip == 1 :
                        second_row.append(trip_dict['{0}_place'.format(idx_trip)])
                    elif idx_trip == 2 :
                        third_row.append(trip_dict['{0}_place'.format(idx_trip)])
                # collect the variables of car counting
                cnt_row.append(var_dict['v_{0}'.format(sa2)])
            result_df =  pd.DataFrame([cnt_row, first_row, second_row, third_row], columns = col_name)    
            categories_df['{0}_{1}'.format(sa, category)] = result_df
            
            writer = pd.ExcelWriter('C://Users//Administrator//Documents//code//data//SA_advanced//analysis_result//{0}_{1}_all_trip.xlsx'.format(sa, category))
            categories_df['{0}_{1}'.format(sa, category)].to_excel(writer,'{0}_{1}_trip'.format(sa, category)) 
            writer.save()
        # saving excel
        #writer = pd.ExcelWriter('C://Users//Administrator//Documents//code//data//SA_advanced//analysis_result//{0}_{1}_all_trip.xlsx'.format(sa, category))
        #writer.save()